In [14]:
import sys

sys.path.append('../')

import faiss
import pickle
from chat.utils import CustomeSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
DATA_DIR = '/Users/arshath/play/chainlink-assistant/data'

In [2]:
with open('/Users/arshath/play/chainlink-assistant/data/chain_link_main_documents.pkl', 'rb') as f:
    main_documents = pickle.load(f)

In [4]:
filtered_main_documents = []
for doc in main_documents:
    if not 'https://chain.link/terms' in doc.metadata['source']:
        filtered_main_documents.append(doc)

In [5]:
for doc in filtered_main_documents:
    if 'https://chain.link/terms' in doc.metadata['source']:
        print(doc.metadata['source'])

In [6]:
with open('/Users/arshath/play/chainlink-assistant/data/chain_link_main_documents.pkl', 'wb') as f:
    pickle.dump(filtered_main_documents, f)

In [7]:
with open('/Users/arshath/play/chainlink-assistant/data/tech_documents.pkl', 'rb') as f:
    tech_documents = pickle.load(f)

with open('/Users/arshath/play/chainlink-assistant/data/blog_documents.pkl', 'rb') as f:
    blog_documents = pickle.load(f)

with open('/Users/arshath/play/chainlink-assistant/data/chain_link_main_documents.pkl', 'rb') as f:
    main_documents = pickle.load(f)

with open('/Users/arshath/play/chainlink-assistant/data/chain_link_you_tube_documents.pkl', 'rb') as f:
    you_tube_documents = pickle.load(f)

with open('/Users/arshath/play/chainlink-assistant/data/education_documents.pkl', 'rb') as f:
    education_documents = pickle.load(f)

with open('/Users/arshath/play/chainlink-assistant/data/stackoverflow_documents.pkl', 'rb') as f:
    stackoverflow_documents = pickle.load(f)

with open('/Users/arshath/play/chainlink-assistant/data/data_documents.pkl', 'rb') as f:
    data_documents = pickle.load(f)

In [9]:
# Create all documents

all_documents = []
all_documents.extend(tech_documents)
all_documents.extend(blog_documents)
all_documents.extend(main_documents)
all_documents.extend(you_tube_documents)
all_documents.extend(education_documents)
all_documents.extend(stackoverflow_documents)

with open('/Users/arshath/play/chainlink-assistant/data/documents.pkl', 'wb') as f:
    pickle.dump(all_documents, f)

In [11]:
for doc in all_documents:
    if 'https://chain.link/terms' in doc.metadata['source']:
        print(doc.metadata['source'])

In [15]:
# Split documents into chunks for 16k model
full_doc_splitter = CustomeSplitter()
chunked_full_documents = full_doc_splitter.split(all_documents)

splitter = RecursiveCharacterTextSplitter(chunk_size=1200, chunk_overlap=50)
split_docs = splitter.split_documents(all_documents)

# Create vectorstore for all documents
vectorstore_all = FAISS.from_documents(split_docs, embedding=OpenAIEmbeddings())

# Split documents into chunks using datadocs
split_docs_data = splitter.split_documents(data_documents)

# Create vectorstore for datadocs
vectorstore_data = FAISS.from_documents(
    split_docs_data, embedding=OpenAIEmbeddings()
)

# Save vectorstores to disk
faiss.write_index(vectorstore_all.index, f"{DATA_DIR}/docs_all.index")
vectorstore_all.index = None
with open(f"{DATA_DIR}/faiss_store_all.pkl", "wb") as f:
    pickle.dump(vectorstore_all, f)

# Save vectorstore_data
faiss.write_index(vectorstore_data.index, f"{DATA_DIR}/docs_data.index")
vectorstore_data.index = None
with open(f"{DATA_DIR}/faiss_store_data.pkl", "wb") as f:
    pickle.dump(vectorstore_data, f)